## Extracting share prices and sending mails weekly with the latest trend and sending mail with significant or specific change in prices

### 1. Import libraries

In [598]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame
from datetime import date, timedelta, datetime
import smtplib
import ssl
from email.message import EmailMessage
import pygsheets

###  2. Make connection with the google sheet
#### (a) Create the Client
#### (b) Enter the name of the downloaded KEYS 
#### (c) File in service_account_file
#### (d) Steps to make a google sheet accessible using pygsheets use the steps mentioned here under the heading "Create Google Service Account" : https://www.analyticsvidhya.com/blog/2020/07/read-and-update-google-spreadsheets-with-python/

In [599]:
client = pygsheets.authorize(service_account_file="clean-patrol-336109-5906eacae9fb.json")

### 3. Access the google sheet and its worksheets

In [600]:
sheet=client.open('demo')
ws1=sheet.worksheet('title','url')

### 4. Access google sheet's data and add data to google sheet
##### 4.1 Collect URLs of organisation's stock price from google sheet and use beautiful soup and requests library to get stock prices

In [601]:
url_df=ws1.get_as_df(include_tailing_empty_rows=False)
organisation_all=[]
date_time=[]
date_time.append(str(date.today()))
for i in range(len(url_df)):
    url=url_df.iloc[i,0]
    r=requests.get(url)
    soup=BeautifulSoup(r.content,'html.parser')
    organisation=soup.title.text
    org_name=organisation.split(',')[0]
    organisation_all.append(org_name)
all1=DataFrame(columns=organisation_all)
all1['Date']=date_time   

In [602]:
for i in range(len(url_df)):
    url=url_df.iloc[i,0]
    r=requests.get(url)
    soup=BeautifulSoup(r.content,'html.parser')
    price=soup.find('div',{'id':'nsecp'}).text 
    change=soup.find('div',{'id':'nsechange'}).text
    organisation_all.append(org_name)
    all1.iloc[0,i]=price
price_all=all1.iloc[:,[2,0,1]]

In [603]:
ws2=sheet.worksheet('title','stock_price')
ws2.update_col(1,['Date'])

##### 4.2 populate stock data in worksheet (keep copy_head = True for first run and then make it False

In [605]:
cell_price='A'+str(len(ws2.get_all_records())+2)
ws2.set_dataframe(price_all,cell_price,copy_head =False)

### 5. Create a bar chart of all data (need to create just once, chart will update automatically after every insertion of data)

##### 5.1 Data for chart:

In [606]:
y_end=len(ws2.get_all_values())
x_end=len(ws2.get_all_values())
cc1=[]
for k in range(len(ws2.get_all_records(majdim='COLUMNS'))):
    y_end2=k+2
    cc2=list([(2,y_end2),(y_end,y_end2)])
    cc1.append(cc2)

##### 5.2 Anchor cell:

In [607]:
AC1=len(ws2.get_all_records(majdim='COLUMNS'))+3
AC=(1,AC1)

##### 5.3 Add bar chart:

In [ ]:
ws2.add_chart(('A2',(x_end,1)),cc1,title='Price',anchor_cell=AC,chart_type=pygsheets.ChartType.BAR)

### 6. Compare today's data with previous 7 day average and generate a mail if today's data fall below the 7 day average

##### 6.1 previous 7 day data

In [614]:
df1=ws2.get_as_df(include_tailing_empty_rows=False)
date_now=max(df1.Date)
date_7=str(((datetime.strptime(max(df1.Date),"%Y-%m-%d")+timedelta(days=-7))).date())
seven_day_data=df1[(df1['Date']>=date_7) & (df1['Date']!=date_now)]

##### 6.2 Check for 7 day average and send mail if today's data is lesser than 7 day average 
##### (To send mail through python first follow the first step mentioned in this webpage : https://towardsdatascience.com/how-to-easily-automate-emails-with-python-8b476045c151 )

In [ ]:
email_sender='input email id of sender'
password='key of google sheet'
email_receiver='input email id of receiver'
for k in range(len(url_df)):
    org_mail=list(df1.columns)[k+1]
    print(org_mail)
    if df1[df1['Date']==date_now].iloc[0,k+1]<seven_day_data.mean(axis=0)[0]:
        print(org_mail+' '+'less than 7 day average')
        subject='todays stocks update'
        body=org_mail +" "+ 'are less than 7 day average'
        em=EmailMessage()
        em['From']=email_sender
        em['To']=email_receiver
        em['Subject']=Subject
        em.set_content(body)
        context=ssl.create_default_context()
        with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
            smtp.login(email_sender, password)
            smtp.sendmail(email_sender, email_receiver, em.as_string())
    else:
        print(org_mail+' '+'Higher than 7 day average')